In [1]:
import pandas as pd
import json
import statistics
import numpy as np
import datetime as dt
import iexfinance as iexf
import pyprind as bar

In [2]:
start = dt.date(2018,5,15)
end = dt.date(dt.date.today().year, dt.date.today().month,dt.date.today().day-1)
end_iexf = dt.date(dt.date.today().year, dt.date.today().month,dt.date.today().day-2)
end_t = dt.datetime(end.year,end.month,end.day-1, 4,0,0)

In [3]:
 stock_symbols = ["BP",
                "FDX",
                "INTC",
                "MSFT",
                "ORCL",
                "UPS",
                "XOM"
                ]
stock_descriptions = {"BP": "BP p.l.c.",
                      "FDX": "FedEx Corporation",
                      "INTC": "Intel Corporation",
                      "MSFT": "Microsoft Corporation",
                      "ORCL": "Oracle Corporation",
                      "UPS": "United Parcel Service Inc.",                      
                      "XOM": "Exxon Mobil Corporation"                      
                     }

#### Yahoo! Finance Data

In [155]:
start_utc = int(dt.datetime(2018,5,15,4,0,0).timestamp())
end_utc = int(end_t.timestamp())

yahoo_url = "https://finance.yahoo.com/quote/{0}/history?period1={1}&period2={2}&interval=1d&filter=history&frequency=1d"

yahoo_col_data = {"Date": "Date",
                  "Open": "Open",
                  "High": "High",
                  "Low": "Low",
                  "Close*": "Close",
                  "Adj Close**": "AdjClose",
                  "Volume": "Volume",
                  "Symbol": "Symbol",
                  "Source": "Source"
}
yahoo_col_dtypes = {"Date": "datetime64[ns]",
                    "Open": "float64",
                    "High": "float64",
                    "Low": "float64",
                    "Close": "float64",
                    "AdjClose": "float64",
                    "Volume": "float64",
                    "Symbol": "object",
                    "Source": "object"
                   }


In [161]:
progress = bar.ProgBar(len(stock_symbols), monitor=True, title="Scraping Yahoo! Finance Historical Stock Performance")
for stock in stock_symbols:
    temp = pd.read_html(yahoo_url.format(stock,start_utc,end_utc))
    df = temp[0][:-1].copy()
    df["Symbol"] = stock
    df["Source"] = "Yahoo!"
    df.rename(columns=yahoo_col_data,inplace=True)
    df.replace(to_replace = "-", value = 0, inplace=True)
    df.drop_duplicates("Date", keep="first", inplace=True)
    df = df.astype(yahoo_col_dtypes)
    df = df.drop(columns="AdjClose")
    df["Date"] = df["Date"].dt.normalize()
    globals()["yahoo_%s" %stock] = df
    globals()["yahoo_%s_df" %stock] = df.set_index("Date")
    globals()["yahoo_%s_df" %stock].index = pd.to_datetime(globals()["yahoo_%s_df" %stock].index)
    globals()["yahoo_%s_df" %stock].sort_index(ascending = True, inplace = True)
    progress.update()

Scraping Yahoo! Finance Historical Stock Performance
0% [#######] 100% | ETA: 00:00:00
Total time elapsed: 00:00:03


#### IEX Financial Data

In [157]:
iex_col_data = ["Date","Open","High","Low","Close","Volume","Symbol","Source"]
iex_col_dtypes = {"Date": "datetime64[ns]",
                  "Open": "float64",
                  "High": "float64",
                  "Low": "float64",
                  "Close": "float64",
                  "Volume": "float64",
                  "Symbol": "object",
                  "Source": "object"
                 }

In [159]:
progress = bar.ProgBar(len(stock_symbols), monitor=True, title="Aggregating IEX Historical Stock Performance")
for stock in stock_symbols:
    globals()["iex_%s" %stock] = iexf.get_historical_data(stock, start=start, end=end_iexf)
    temp_date = []
    temp_open = []
    temp_high = []
    temp_low = []
    temp_close = []
    temp_vol = []
    temp_symbol = []
    temp_source = []
    

    temp_keys = globals()["iex_%s" %stock][stock]
    for key in temp_keys.keys():
        temp_date.append(key)
        temp_open.append(temp_keys[key]["open"])
        temp_high.append(temp_keys[key]["high"])
        temp_low.append(temp_keys[key]["low"])
        temp_close.append(temp_keys[key]["close"])
        temp_vol.append(temp_keys[key]["volume"])
        temp_symbol.append(stock)
        temp_source.append("IEX")
    globals()["iex_%s" %stock] = pd.DataFrame(np.column_stack([temp_date,
                                                               temp_open, 
                                                               temp_high,
                                                               temp_low,
                                                               temp_close,
                                                               temp_vol,
                                                               temp_symbol,
                                                               temp_source]),
                          columns=iex_col_data).astype(iex_col_dtypes)
    globals()["iex_%s_df" %stock] = globals()["iex_%s" %stock].set_index(["Date"])
    progress.update()


Aggregating IEX Historical Stock Performance
0% [#######] 100% | ETA: 00:00:00
Total time elapsed: 00:00:01


##### Yahoo! Finance DataFrames Type and Index Validation

In [162]:
for stock in stock_symbols:
    total = len(globals()["yahoo_%s_df" %stock])
    print("yahoo_%s_df: \n*******************" %stock)
    print(str(globals()["yahoo_%s_df" %stock].dtypes)+
          "\n___________________\nINDEX\n-------------------\n"+
          str((globals()["yahoo_%s_df" %stock].index.name))+": "+
          (globals()["yahoo_%s_df" %stock].index.dtype_str)+
          "\nTotal Records: "+str(total)+"\n\n")
    
print("Total Stocks Captured: "+str(len(stock_symbols)))

yahoo_BP_df: 
*******************
Open      float64
High      float64
Low       float64
Close     float64
Volume    float64
Symbol     object
Source     object
dtype: object
___________________
INDEX
-------------------
Date: datetime64[ns]
Total Records: 30


yahoo_FDX_df: 
*******************
Open      float64
High      float64
Low       float64
Close     float64
Volume    float64
Symbol     object
Source     object
dtype: object
___________________
INDEX
-------------------
Date: datetime64[ns]
Total Records: 30


yahoo_INTC_df: 
*******************
Open      float64
High      float64
Low       float64
Close     float64
Volume    float64
Symbol     object
Source     object
dtype: object
___________________
INDEX
-------------------
Date: datetime64[ns]
Total Records: 30


yahoo_MSFT_df: 
*******************
Open      float64
High      float64
Low       float64
Close     float64
Volume    float64
Symbol     object
Source     object
dtype: object
___________________
INDEX
------------

##### IEX Financial DataFrames Type and Index Validation

In [163]:
for stock in stock_symbols:
    total = len(globals()["iex_%s_df" %stock])
    print("iex_%s_df: \n*******************" %stock)
    print(str(globals()["iex_%s_df" %stock].dtypes)+
          "\n___________________\nINDEX\n-------------------\n"+
          str((globals()["iex_%s_df" %stock].index.name))+": "+
          (globals()["iex_%s_df" %stock].index.dtype_str)+
          "\nTotal Records: "+str(total)+"\n\n")
    
print("Total Stocks Captured: "+str(len(stock_symbols)))

iex_BP_df: 
*******************
Open      float64
High      float64
Low       float64
Close     float64
Volume    float64
Symbol     object
Source     object
dtype: object
___________________
INDEX
-------------------
Date: datetime64[ns]
Total Records: 30


iex_FDX_df: 
*******************
Open      float64
High      float64
Low       float64
Close     float64
Volume    float64
Symbol     object
Source     object
dtype: object
___________________
INDEX
-------------------
Date: datetime64[ns]
Total Records: 30


iex_INTC_df: 
*******************
Open      float64
High      float64
Low       float64
Close     float64
Volume    float64
Symbol     object
Source     object
dtype: object
___________________
INDEX
-------------------
Date: datetime64[ns]
Total Records: 30


iex_MSFT_df: 
*******************
Open      float64
High      float64
Low       float64
Close     float64
Volume    float64
Symbol     object
Source     object
dtype: object
___________________
INDEX
-------------------


#### Consolidating each IEX and Yahoo! Dataframes into one per stock

In [164]:
for stock in stock_symbols:
    yahoo = globals()["yahoo_%s_df" %stock]
    iex = globals()["iex_%s_df" %stock]
    globals()["%s_df" %stock] = pd.concat([yahoo,iex], sort=False)

#### Combining all Stock Dataframes into one Dataframe

In [165]:
for stock in stock_symbols:
    
Stocks_df = pd.concat([BP_df,FDX_df,INTC_df,MSFT_df,ORCL_df,UPS_df,XOM_df])

In [166]:
test = Stocks_df

In [167]:
type(test.index[5])

pandas._libs.tslibs.timestamps.Timestamp

#### Assigning the categories of the Summit timeline to the stock data

In [132]:
events = { "Coordination": [dt.date(2018,5,15),
                            dt.date(2018,5,23)],
          
           "Cancelled":    [dt.date(2018,5,24)],
          
           "Rescheduled":  [dt.date(2018,5,25),
                            dt.date(2018,6,1)],
           
           "Awaiting":     [dt.date(2018,6,2),
                            dt.date(2018,6,11)],
          
           "Summit":       [dt.date(2018,6,12)],
           "Aftermath":    [dt.date(2018,6,13)]
         }

In [133]:
stockDateEvent = pd.DataFrame(np.column_stack([Stocks_df.index,Stocks_df["Event"]]),
                          columns=["Date", "Event"]).astype({"Date": "datetime64[ns]","Event": "object"})


In [134]:
stockDateEvent.drop_duplicates(inplace=True)

In [135]:
temp_date_map = []
temp_event_map = []
for x in range(len(stockDateEvent)):
    e_date = stockDateEvent.Date[x].date()
    temp_date_map.append(e_date)
    if e_date == events["Summit"][0]:
        temp_event_map.append("Summit")
    elif e_date == events["Cancelled"][0]:
        temp_event_map.append("Cancelled")
    elif events["Coordination"][0] <= e_date <= events["Coordination"][1]:
        temp_event_map.append("Coordination")
    elif events["Rescheduled"][0] <= e_date <= events["Rescheduled"][1]:
        temp_event_map.append("Rescheduled")
    elif events["Awaiting"][0] <= e_date <= events["Awaiting"][1]:
        temp_event_map.append("Awaiting")
    else:
        temp_event_map.append("Aftermath")


In [176]:
dateJoin = pd.DataFrame(np.column_stack([temp_date_map,temp_event_map]), columns=["Date","Event"]).astype({"Date": "datetime64[ns]","Event": "object"})
eventMap = dateJoin.set_index("Date")
mapper = eventMap.to_dict("index")

In [181]:
stockAnalysis = pd.DataFrame(Stocks_df.merge(eventMap, left_index=True, right_index=True, how="left"))
stockAnalysis.to_csv("stockAnalysis.csv")